In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import pymysql
import pickle

In [2]:
con = pymysql.connect(host="127.0.0.1", db="riot", user="root", password="root1234", charset='utf8')
middata = pd.read_sql("select * from middata", con)
middata

D:\kdigital\anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,mno,blue_mid_champ,bm_kill,bm_gold,bm_damageDone,bm_damageTaken,bm_level,bm_minion,red_mid_champ,rm_kill,rm_gold,rm_damageDone,rm_damageTaken,rm_level,rm_minion,result
0,1,Diana,4,7820,5021,8904,11,20,Akali,2,5643,9259,9340,11,116,1
1,2,Anivia,6,7553,9842,7934,10,110,Vex,0,4675,5821,6926,10,95,0
2,3,Gragas,4,7862,7985,8605,11,105,Yasuo,2,5403,7897,11894,9,98,0
3,4,Pantheon,1,4750,7368,9954,10,100,Yasuo,9,8265,12570,8821,12,119,1
4,5,Viktor,1,5062,5872,6937,10,117,Zed,3,5409,6437,6358,11,105,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,29545,Sylas,1,4706,3410,11690,9,89,Galio,8,7402,10240,8162,11,100,1
29544,29546,Galio,2,4930,4648,6607,10,96,Qiyana,1,5163,7461,6024,9,101,0
29545,29547,Sylas,5,5825,7513,10082,11,93,Yone,0,4160,6999,9259,9,88,1
29546,29548,Lissandra,0,4559,4035,7695,10,89,Ahri,1,5176,7584,9137,10,97,1


In [3]:
# 데이터 타입 확인
middata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29548 entries, 0 to 29547
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   mno             29548 non-null  int64 
 1   blue_mid_champ  29548 non-null  object
 2   bm_kill         29548 non-null  int64 
 3   bm_gold         29548 non-null  int64 
 4   bm_damageDone   29548 non-null  int64 
 5   bm_damageTaken  29548 non-null  int64 
 6   bm_level        29548 non-null  int64 
 7   bm_minion       29548 non-null  int64 
 8   red_mid_champ   29548 non-null  object
 9   rm_kill         29548 non-null  int64 
 10  rm_gold         29548 non-null  int64 
 11  rm_damageDone   29548 non-null  int64 
 12  rm_damageTaken  29548 non-null  int64 
 13  rm_level        29548 non-null  int64 
 14  rm_minion       29548 non-null  int64 
 15  result          29548 non-null  int64 
dtypes: int64(14), object(2)
memory usage: 3.6+ MB


In [4]:
# champ code 불러오기
with open('champ.pickle','rb') as f:
    champCode = pickle.load(f)

champCode[['champion_key','en_name']]

,champion_key,en_name
0,266,Aatrox
1,103,Ahri
2,84,Akali
3,166,Akshan
4,12,Alistar
...,...,...
156,221,Zeri
157,115,Ziggs
158,26,Zilean
159,142,Zoe


In [5]:
# 챔피언명 라벨링
def changeCode(x):
    for code, name in champCode[['champion_key','en_name']].iterrows():
        if x == name[1]:
            return name[0]
red_mid_champ_code = middata['red_mid_champ'].apply(changeCode)
blue_mid_champ_code = middata['blue_mid_champ'].apply(changeCode)

chmapToCode = pd.concat([red_mid_champ_code,blue_mid_champ_code],axis=1)
chmapToCode

,red_mid_champ,blue_mid_champ
0,84,131
1,711,34
2,157,79
3,157,80
4,238,112
...,...,...
29543,3,517
29544,246,3
29545,777,517
29546,103,127


In [7]:
#라벨링된 데이터로변경
labelMid = pd.concat([chmapToCode,middata.drop(['blue_mid_champ','red_mid_champ'], axis=1)],axis=1)

#결측치 제거
labelMid = labelMid.dropna(axis=0)

#타입변경
labelMid = labelMid.astype({'red_mid_champ':'int64'})
labelMid.info()
labelMid

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29490 entries, 0 to 29547
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   red_mid_champ   29490 non-null  int64 
 1   blue_mid_champ  29490 non-null  object
 2   mno             29490 non-null  int64 
 3   bm_kill         29490 non-null  int64 
 4   bm_gold         29490 non-null  int64 
 5   bm_damageDone   29490 non-null  int64 
 6   bm_damageTaken  29490 non-null  int64 
 7   bm_level        29490 non-null  int64 
 8   bm_minion       29490 non-null  int64 
 9   rm_kill         29490 non-null  int64 
 10  rm_gold         29490 non-null  int64 
 11  rm_damageDone   29490 non-null  int64 
 12  rm_damageTaken  29490 non-null  int64 
 13  rm_level        29490 non-null  int64 
 14  rm_minion       29490 non-null  int64 
 15  result          29490 non-null  int64 
dtypes: int64(15), object(1)
memory usage: 3.8+ MB


,red_mid_champ,blue_mid_champ,mno,bm_kill,bm_gold,bm_damageDone,bm_damageTaken,bm_level,bm_minion,rm_kill,rm_gold,rm_damageDone,rm_damageTaken,rm_level,rm_minion,result
0,84,131,1,4,7820,5021,8904,11,20,2,5643,9259,9340,11,116,1
1,711,34,2,6,7553,9842,7934,10,110,0,4675,5821,6926,10,95,0
2,157,79,3,4,7862,7985,8605,11,105,2,5403,7897,11894,9,98,0
3,157,80,4,1,4750,7368,9954,10,100,9,8265,12570,8821,12,119,1
4,238,112,5,1,5062,5872,6937,10,117,3,5409,6437,6358,11,105,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,3,517,29545,1,4706,3410,11690,9,89,8,7402,10240,8162,11,100,1
29544,246,3,29546,2,4930,4648,6607,10,96,1,5163,7461,6024,9,101,0
29545,777,517,29547,5,5825,7513,10082,11,93,0,4160,6999,9259,9,88,1
29546,103,127,29548,0,4559,4035,7695,10,89,1,5176,7584,9137,10,97,1


In [8]:
middata = labelMid.drop('mno',axis=1)
middata

,red_mid_champ,blue_mid_champ,bm_kill,bm_gold,bm_damageDone,bm_damageTaken,bm_level,bm_minion,rm_kill,rm_gold,rm_damageDone,rm_damageTaken,rm_level,rm_minion,result
0,84,131,4,7820,5021,8904,11,20,2,5643,9259,9340,11,116,1
1,711,34,6,7553,9842,7934,10,110,0,4675,5821,6926,10,95,0
2,157,79,4,7862,7985,8605,11,105,2,5403,7897,11894,9,98,0
3,157,80,1,4750,7368,9954,10,100,9,8265,12570,8821,12,119,1
4,238,112,1,5062,5872,6937,10,117,3,5409,6437,6358,11,105,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,3,517,1,4706,3410,11690,9,89,8,7402,10240,8162,11,100,1
29544,246,3,2,4930,4648,6607,10,96,1,5163,7461,6024,9,101,0
29545,777,517,5,5825,7513,10082,11,93,0,4160,6999,9259,9,88,1
29546,103,127,0,4559,4035,7695,10,89,1,5176,7584,9137,10,97,1


## 데이터분석

In [9]:
from sklearn.model_selection import train_test_split

In [51]:
x_train, x_test, y_train, y_test=train_test_split(middata.drop('result',axis=1), middata['result'], test_size=0.25)
len(x_train), len(x_test), len(y_train), len(y_test)

(22117, 7373, 22117, 7373)

In [52]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', random_state=1, C=0.2)
model.fit(x_train, y_train)
pred = model.predict(x_test)
(pred==y_test).mean()


0.6792350467923505

In [53]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
accuracy_score(pred,y_test),precision_score(pred,y_test),recall_score(pred,y_test)

(0.6792350467923505, 0.6720430107526881, 0.6858710562414266)